In [6]:
import os
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv("api_key")
langsmith_api_key = os.getenv("langsmith_key")
weather_api_key = os.getenv("weather_api_key")
groq_api_key = os.getenv("groq_api_key")

os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['LANGSMITH_API_KEY'] = langsmith_api_key
os.environ['LANGSMITH_TRACING'] = "true"
os.environ['LANGCHAIN_PROJECT'] = "Multi-agent"
os.environ['key'] = weather_api_key

In [7]:
from typing import TypedDict, List

class GraphState_querygpt(TypedDict):

    user_question: str
    intent_agent_output: str
    table_agent_output: List[str]
    column_agent_output: List[str]
    is_query_execution_possible : bool
    query: str
    query_output: str

In [8]:
# wikipedia retirval 

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=1 , doc_content_chars_max=300)
wiki = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)
print(wiki.run("LangChain"))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis a


In [11]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq

class source_decider (BaseModel):
    used_tool : Literal['wikipedia', 'custom_database'] = Field(
        ...,
        description=" Given the user question choose the router to take the when the question is very generic go with  wikipedia or when question is realted to anything which would required custom database go with custom_database"
    )



LLM_1 = ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
structured_LLM_source_router = LLM_1.with_structured_output(source_decider)

system_prompt = """You are an expert at routing a user question to a wikipedia or custom_database related to products, suppliers and all .
Use the custom_database for questions on the topics like related to products, suppliers related. Otherwise, use wikipedia"""

router_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{question}"),
    ]
)

final_source_router = router_prompt | structured_LLM_source_router 


final_source_router.invoke({"question" : "Who is Shahruk khan"})
final_source_router.invoke({"question" : "what is the highest amount of products sold?"})


def intent_agent(state : GraphState_querygpt) -> GraphState_querygpt:

    print("The intent Agent to find the source of the user question")
    
    user_question = state['user_question']
    output_source = final_source_router.invoke({"question" : {user_question}})

    if output_source.used_tool == "wikipedia":
        return "use_wikipedia"
    elif output_source.used_tool == "custom_database":
        return "use_custom_database"


source_decider(used_tool='custom_database')